<a href="https://colab.research.google.com/github/Thabet-Chaaouri/Transformers/blob/main/Fine_tuning_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.0 MB/s 
     |████████████████████████████████| 182 kB 64.8 MB/s 
     |████████████████████████████████| 132 kB 48.2 MB/s 
     |████████████████████████████████| 212 kB 27.1 MB/s 
     |████████████████████████████████| 127 kB 26.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.1 MB/s 
     |████████████████████████████████| 7.6 MB 69.1 MB/s 


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.1 MB/s 


In [ ]:
from transformers import TFAutoModelForMaskedLM

checkpoint="distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(checkpoint)


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [ ]:
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [ ]:
text = "This is a great [MASK]."

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import numpy as np

inputs=tokenizer(text, return_tensors="tf")
mask_token_index=np.argwhere(inputs["input_ids"]==tokenizer.mask_token_id)[0,1]
token_logits=model(**inputs).logits
mask_token_logits  = token_logits[0,mask_token_index,:]
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

In [ ]:
[tokenizer.decode(token) for token in top_5_tokens]

['deal', 'success', 'adventure', 'idea', 'feat']

In [ ]:
from datasets import  load_dataset

imdb_dataset = load_dataset("imdb")

imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
def tokenize_function(examples):
  result= tokenizer(examples["text"])
  if tokenizer.is_fast:
    result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
  return result

In [ ]:
tokenized_datasets=imdb_dataset.map(tokenize_function, batched=True, remove_columns=["text", "label"])
tokenized_datasets

  0%|          | 0/25 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [ ]:
chunk_size = tokenizer.model_max_length

In [ ]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // chunk_size) * chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 15313
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 14966
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 30721
    })
})

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'the brown bunny, in which we\'re treated to the site of vincent gallo\'s throbbing johnson, but not a trace of pink visible on chloe sevigny. before crying ( or implying ) " double - standard " in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women : there are no genitals on display when actresses appears nude, and the same cannot be said for a man. in fact, you generally won\'t see female genitals in an american film in anything short of porn or explicit erotica. this alleged double - standard is less a double standard than an admittedly depressing ability to come to terms culturally with the insides of women\'s bodies. [SEP] [CLS] if only to avoid making this type of film in the future. this film is interesting as an experiment but tells no cogent story. < br / > < br / > one might feel virtuous for sitting thru it because it touches on so many important issues but it does so without any discernable moti

In [ ]:
tokenizer.decode(lm_datasets["train"][2]["input_ids"])

'" is that all there is?? "... i was just an early teen when this smoked fish hit the u. s. i was too young to get in the theater ( although i did manage to sneak into " goodbye columbus " ). then a screening at a local film museum beckoned - finally i could see this film, except now i was as old as my parents were when they schlepped to see it!! < br / > < br / > the only reason this film was not condemned to the anonymous sands of time was because of the obscenity case sparked by its u. s. release. millions of people flocked to this stinker, thinking they were going to see a sex film... instead, they got lots of closeups of gnarly, repulsive swedes, on - street interviews in bland shopping malls, asinie political pretension... and feeble who - cares simulated sex scenes with saggy, pale actors. < br / > < br / > cultural icon, holy grail, historic artifact.. whatever this thing was, shred it, burn it, then stuff the ashes in a lead box! < br / > < br / > elite esthetes still scrape t

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i [MASK] i am curious - yellow [MASK] my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard [MASK] at first it was seized by u. s [MASK] customs if it [MASK] tried to [MASK] this country [MASK] therefore being a fan of films considered " controversial " [MASK] really had to see thisnl myself. < br / > < br / > the plot is centered [MASK] a young [MASK] drama student named lena who wants to [MASK] everything she can about life. in particular she wants to focus her attentions to [MASK] some sort of [MASK] on what the average sw [MASK] thought about certain political [MASK] such as the vietnam war and race [MASK] in the united states. in between asking [MASK] and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama [MASK], classmates, [MASK] married men. < br / > < br / > what kills me about i am curious - yellow [MASK] [MASK] [MASK] years ago, this was considered [MASK]. really 

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base",use_fast=True)

ValueError: ignored

In [ ]:
tokenizer.model_max_length